In [13]:
#sudo pip install OsmApi
#sudo pip install geopy
from geopy.geocoders import Nominatim
from geopy.distance import vincenty
from osmapi import OsmApi
import json

#https://wiki.openstreetmap.org/wiki/Nominatim
geolocator = Nominatim()
location = geolocator.reverse("38.8105401, -77.4889331")
MyApi = OsmApi()
print location


Custis Memorial Parkway, Fairfax County, Virginia, 20155, United States of America


In [52]:
print MyApi.NodeGet(8820105)
point1 = (38.8105401,-77.4889331)
point2 = (38.8105087,-77.4889102)
print(vincenty(point1, point2).miles)

None
0.00249376602913


In [6]:
#Use http://boundingbox.klokantech.com/ to get bounding box co-ordinates. Use OsmApi.Map which returns list of dicts. 
abc = (MyApi.Map(-77.4905548096,38.810092926,-77.4899978638,38.8103103638))


''


In [51]:
print len(abc)
import yaml

#first_tag = abc[200]
#http://wiki.openstreetmap.org/wiki/Key:highway

for a in abc:
    if a['type'] == "way":
        print "Printing way"
        print a['data']['tag']

605
Printing way
{u'tiger:name_base': u'I-66', u'horse': u'no', u'maxspeed': u'60 mph', u'bicycle': u'no', u'name': u'Custis Memorial Parkway', u'tiger:cfcc': u'A15', u'tiger:separated': u'yes', u'hgv': u'designated', u'tiger:source': u'tiger_import_dch_v0.6_20070830', u'hov': u'lane', u'note:lanes': u'left lane is hov', u'tiger:county': u'Prince William, VA', u'tiger:tlid': u'207164940:207165331:207165332:207165350:207165353:207165368:207165369:20717:207170088:207170090:207172262:207172283:207174164:207174178:207174180:207174637:207174696:207174862:207176742:207176934:207177297:207178281:207183096:207183099:207183399:207183448', u'oneway': u'yes', u'foot': u'no', u'lanes': u'4', u'sidewalk': u'none', u'ref': u'I 66', u'highway': u'motorway', u'tiger:upload_uuid': u'bulk_upload.pl-3a1ec051-2146-4ec9-9e72-455377b871a7'}
Printing way
{u'water': u'river', u'natural': u'water'}
Printing way
{u'bridge': u'yes', u'horse': u'no', u'maxspeed': u'60 mph', u'bicycle': u'no', u'name': u'Custis Me

In [138]:
#http://manishamde.github.io/blog/2013/03/07/pandas-and-python-top-10/
import pandas as pd
inven = pd.read_csv('/home/sudeepgaddam/data_science/labs/nist/data/detector_lane_inventory.csv')
print inven[:0]
inven =  inven.ix[:,['direction','zone_id','latitude','longitude']]
#print inven
east_inven = inven[inven['direction']=='East']
west_inven = inven[inven['direction']=='West']
north_inven = inven[inven['direction']=='North']
south_inven = inven[inven['direction']=='South']

#grouped = east_inven['latitude','longitude'].groupby(east_inven['zone_id'])
grouped  = east_inven.groupby(['zone_id'], as_index=False).mean()
print grouped




Empty DataFrame
Columns: [lane_id, zone_id, lane_number, name, state, road, direction, location_description, lane_type, organization, detector_type, latitude, longitude, bearing, default_speed, interval]
Index: []

[0 rows x 16 columns]
    zone_id   latitude  longitude
0        40  39.384129 -76.523505
1        41  39.390327 -76.714290
2        42  39.379282 -76.519755
3        55  39.296079 -76.855962
4        57  39.401979 -77.448155
5       101  39.020399 -76.477562
6       103  38.983888 -76.588853
7       106  39.014080 -76.491367
8       108  38.960735 -76.654701
9       110  38.993769 -76.515234
10      112  38.945252 -76.770554
11      113  38.946884 -76.821862
12      114  38.954903 -76.716926
13      118  39.018323 -76.411881
14      802  38.886009 -77.219026
15      803  38.900451 -77.190584
16      804  38.884749 -77.153754
17      808  38.886737 -77.117281
18      817  38.896858 -77.083793
19      822  38.879477 -77.137758
20      827  38.889226 -77.163928
21      830  38

In [157]:
grouped['latlong'] = grouped.apply(lambda r: (r['latitude'], r['longitude']), axis=1)

#east_lat_long =  east_inven['latlong']
east_lat_long =  grouped.ix[:,['latlong','zone_id']]
east_lat_long['dummy'] = 0
joined = pd.merge(east_lat_long,east_lat_long,on='dummy')
del joined['dummy']
#Before executing below condition: 133956
joined = joined[joined.zone_id_x < joined.zone_id_y]
joined['distance'] = joined.apply(lambda r: (vincenty(r['latlong_x'], r['latlong_y']).miles ), axis=1)
joined =  joined.ix[:,['zone_id_x','zone_id_y','distance','latlong_x']]

In [158]:
#print joined
joined = joined[joined.distance < 20]
print joined[joined.zone_id_x == 40]
print joined[joined.zone_id_x == 41]

    zone_id_x  zone_id_y   distance                latlong_x
1          40         41  10.222148  (39.384129, -76.523505)
2          40         42   0.390017  (39.384129, -76.523505)
3          40         55  18.816599  (39.384129, -76.523505)
76         40       3203   1.492423  (39.384129, -76.523505)
77         40       3204   1.492423  (39.384129, -76.523505)
78         40       3206   0.987361  (39.384129, -76.523505)
79         40       3208   7.918008  (39.384129, -76.523505)
80         40       3209   7.918008  (39.384129, -76.523505)
81         40       3221   1.985996  (39.384129, -76.523505)
82         40       3224   7.208520  (39.384129, -76.523505)
83         40       3225   7.208520  (39.384129, -76.523505)
84         40       3246   6.409530  (39.384129, -76.523505)
85         40       3251   5.344032  (39.384129, -76.523505)
86         40       3252   5.344032  (39.384129, -76.523505)
87         40       3257   3.137377  (39.384129, -76.523505)
88         40       3258